In [46]:
import cv2
import mediapipe as mp
import os

In [3]:
mp_draw = mp.solutions.drawing_utils
mp_face_detection = mp.solutions.face_detection

In [38]:
# Initiate facedetection model
face_detection = mp_face_detection.FaceDetection(model_selection = 0, min_detection_confidence = 0.7)

In [217]:
def get_bboxs(frame):
    mp_detections = face_detection.process(frame)
    score_bboxs = []
    if mp_detections.detections:
        for detection in mp_detections.detections:
            score = detection.score[0]
            mp_bbox = detection.location_data.relative_bounding_box
            bbox_dict = {
                'x_min': mp_bbox.xmin,
                'y_min': mp_bbox.ymin,
                'w': mp_bbox.width,
                'h': mp_bbox.height
            }

            score_bboxs.append([score, bbox_dict])

    return score_bboxs

In [223]:
def draw_bbox(score, bbox_dict, frame, col = (255, 0, 255)):
    x_min, y_min, w, h = bbox_dict.values()
    frame_h, frame_w, _ = frame.shape
    bbox = int(x_min * frame_w), int(y_min * frame_h), int(w * frame_w), int(h * frame_h)

    # draw bbox
    cv2.rectangle(frame, bbox, col, 2)
    cv2.putText(frame, str(round(score, 3)), (bbox[0], bbox[1] - 10),
                cv2.FONT_HERSHEY_COMPLEX, 0.6, col, 1)

In [227]:
# MAIN
# param is 0, first webcam in list of webcams
cam = cv2.VideoCapture(0)

while cam.isOpened():
    # take frame from webcam
    success, frame = cam.read()
    # flip frame for selfie mode
    frame = cv2.flip(frame, 1)
    # check if loaded frame
    if not success:
        print("Image is donezo gonezo")
        continue

    # frame setup
    frame.flags.writeable = False # not writeable, pass by reference
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # rgb for mediapipe use
    # detect boxes
    bboxs = get_bboxs(frame)

    # draw bbox on frame
    frame.flags.writeable = True # now need to draw on
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR) # back to bgr for opencv

    if bboxs:
        for bbox in bboxs:
            #Draw box on face
            draw_bbox(bbox[0], bbox[1], frame)

    # display frame
    cv2.imshow('Webcam', frame)

    # use q to quit
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# close the camera
cam.release()
# Close windows
cv2.destroyAllWindows()